In [1]:
#Question Answering system using a pdf\
# Below is testing code

In [1]:
#Install the libraries
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchain_openai
!pip install -q openai
!pip install -q langchain_core
!pip install -q pypdf
!pip install -q chromadb
!pip install -q Flask
!pip install -q markdown
!pip install -q langchain guardrails-ai
!pip install -q guardrails-ai



In [2]:
# configure with api key
!guardrails configure

Enable anonymous metrics reporting? [Y/n]: ^C
Aborted.


In [3]:
# install guardrails libraries
!guardrails hub install hub://guardrails/ban_list
!guardrails hub install hub://guardrails/bias_check
!guardrails hub install hub://guardrails/nsfw_text
!guardrails hub install hub://guardrails/profanity_free
!guardrails hub install hub://guardrails/logic_check
!guardrails hub install hub://cartesia/mentions_drugs
!guardrails hub install hub://guardrails/politeness_check
!guardrails hub install hub://guardrails/toxic_language

Installing hub://guardrails/ban_list...
[====] Fetching manifestst
[   =] Downloading dependenciespendencies
[    ] Running post-install setup
✅Successfully installed guardrails/ban_list!


Import validator:
from guardrails.hub import BanList

Get more info:
https://hub.guardrailsai.com/validator/guardrails/ban_list

Installing hub://guardrails/bias_check...
[   =] Fetching manifestst
[====] Downloading dependenciespendencies
[==  ] Running post-install setuptall setup2025-01-11 13:22:09.148117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[=== ] Running post-install setupAll model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the mode

In [6]:
##Imports for Baseline QA Pipeline
from langchain.document_loaders import PyPDFLoader # for loading the pdf
from langchain_openai import OpenAIEmbeddings # for creating embeddings
from langchain.vectorstores import Chroma # for the vectorization part
from langchain.chains import RetrievalQA #For the retrieval QA chain part # apparently deprecated
from langchain_openai import ChatOpenAI #for getting an LLM for QA chain
#from langchain_core.output_parsers import StrOutputParser #Not used currently, leaving, as can be used for parsing output from LLM
#from langchain_core.runnables import RunnablePassthrough #Not used currently, leaving, as can be used for getting LLM output
from langchain.prompts import ChatPromptTemplate #for setting up prompts


#Guardrails stuff
from guardrails import Guard, OnFailAction
from guardrails.hub import BanList, BiasCheck, NSFWText, ProfanityFree, LogicCheck, MentionsDrugs, PolitenessCheck #, ToxicLanguage# Updated import
#import guardrails.hub #which needs fuzzysearch, which is already downloaded
#from guardrails.datatypes import String
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [4]:
#Setup openai key
import os
import openai
from getpass import getpass
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
#Download a sample pdf
!curl https://www.mrbigler.com/downloads/Notes-Physics-1.pdf >Notes_Physics.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.8M  100 10.8M    0     0  11.9M      0 --:--:-- --:--:-- --:--:-- 11.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    69    0    69    0     0    164      0 --:--:-- --:--:-- --:--:--   164


In [ ]:

from guardrails import Guard


guard = Guard().use_many(
    BiasCheck(
        threshold=0.5,
        on_fail="exception"
    ),

    NSFWText(
        threshold=0.8,
        validation_method="sentence"
    ),

    ProfanityFree(
        on_fail = "exception"
    ),

    LogicCheck(
        model="gpt-3.5-turbo",
        on_fail="exception"
    ),

    MentionsDrugs(
        on_fail = "exception"
    ),

    PolitenessCheck(
        llm_callable="gpt-3.5-turbo",
        on_fail = "exception"
    ),

    ToxicLanguage(
        threshold=0.5,
        validation_method="sentence",
        on_fail="exception"
    )

)

try:
  guard.validate("I hate you!")
  print("hello")
except Exception as e:
  print(e)

In [ ]:
#Setup Base QA system pipeline
class BaseQAPipeline:
    def __init__(self):
        self.doc = "tutor_textbook.pdf"
        self.loader = PyPDFLoader(self.doc)

        # Load the document and store it in the 'data' variable
        self.data = self.loader.load_and_split()

        self.embeddings = OpenAIEmbeddings()
        self.vectordb = Chroma.from_documents(self.data, embedding=self.embeddings,
                                 persist_directory=".")

        # Initialize a language model with ChatOpenAI
        self.llm = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.6)

        #Setup a prompt template
        template = """\
        You are an assistant for question-answering tasks.

        Use the following pieces of retrieved context to answer the question.

        If either the PDF or the question are not related to each other or not 
        related to any educational standard, state the following: This content is 
        not related to any educational purposes. 

        For example, if topics are not the same, like a java textbook is given, 
        however, the user asks about a physics question, state the following: This
        content is not related to the inputted textbook, please select another textbook
        and try again.

        If you don't know the answer, just say that you don't know.

        Use three sentences maximum and keep the answer concise. 

        Question: {question}

        Context: {context}

        Answer:

        """

        prompt = ChatPromptTemplate.from_template(template)

        chain_type_kwargs = {"prompt": prompt}



        # 1. Vectorstore-based retriever
        self.vectorstore_retriever = self.vectordb.as_retriever()

        # Initialize a RetrievalQA chain with the language model and vector database retriever
        self.qa_chain = RetrievalQA.from_chain_type(self.llm, retriever= self.vectorstore_retriever, chain_type_kwargs=chain_type_kwargs)
        self.chat_history = []  # Initialize chat history

    def update_chat_history(self, question, answer):
        self.chat_history.append({"question": question, "answer": answer})

    def build_combined_context(self):
        """Combine chat history and document context."""
        # Combine all previous chat history
        chat_context = "\n".join([f"Q: {entry['question']}\nA: {entry['answer']}" for entry in self.chat_history])
        
        # Fetch relevant context from the vector store based on the current question
        if self.chat_history:
            current_question = self.chat_history[-1]['question']
            context_from_db = self.vectorstore_retriever.get_relevant_documents(current_question)
        else:
            context_from_db = self.vectorstore_retriever.get_relevant_documents("")

        # Convert the list of context documents into a string
        context_str = "\n".join([doc.page_content for doc in context_from_db])

        # Combine both chat history and the document context
        combined_context = f"Chat history:\n{chat_context}\n\nContext from the document:\n{context_str}"
        
        return combined_context


    def invoke(self, input_dict):
        question = input_dict.get("question")
        #here put a if-else that returns true or false depending on if question passes
        # guardrails checks
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"
        
        combined_context = self.build_combined_context()

        result = self.qa_chain.invoke({
            "query": question,
            "context": combined_context
        })

        self.update_chat_history(question, result['result'])
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"
        else:
          return result

    def guardrails(self, input):
      #if guardrails return true send back whatever the input is,
      #else send back an error message
      try:
        guard.validate(input)
        return True
      except Exception as e:
        return False

In [ ]:
#Setup GenerateStudyPlan pipeline
class GenerateStudyPlan:
    def __init__(self):
        self.doc = "tutor_textbook.pdf"
        self.loader = PyPDFLoader(self.doc)

        # Load the document and store it in the 'data' variable
        self.data = self.loader.load_and_split()

        self.embeddings = OpenAIEmbeddings()
        self.vectordb = Chroma.from_documents(self.data, embedding=self.embeddings,
                                 persist_directory=".")

        # Initialize a language model with ChatOpenAI
        self.llm = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.6)

        #Setup a prompt template
        template = """\
            You are an assistant for generating study plans on a singular subject.

        Use the following pieces of retrieved context to answer the question.

        If the user has given a topic to study or topics that they need focus on,
        make the plan more focused on those topics.

        Give a nice and detailed study plan. If the user doesnt specify a type of 
        study plan (schedule wise), you can come up with your own, hourly or daily 
        plan, or you can ask the user to give the same instructions but with the 
        study plan of their choice. 

        Question: {question}

        Context: {context}

        Answer:

        """

        prompt = ChatPromptTemplate.from_template(template)

        chain_type_kwargs = {"prompt": prompt}


        # 1. Vectorstore-based retriever
        self.vectorstore_retriever = self.vectordb.as_retriever()

        # Initialize a RetrievalQA chain with the language model and vector database retriever
        self.qa_chain = RetrievalQA.from_chain_type(self.llm, retriever= self.vectorstore_retriever, chain_type_kwargs=chain_type_kwargs)
        self.chat_history = []  # Initialize chat history

    def update_chat_history(self, question, answer):
        self.chat_history.append({"question": question, "answer": answer})

    def build_combined_context(self):
        """Combine chat history and document context."""
        # Combine all previous chat history
        chat_context = "\n".join([f"Q: {entry['question']}\nA: {entry['answer']}" for entry in self.chat_history])
        
        # Fetch relevant context from the vector store based on the current question
        if self.chat_history:
            current_question = self.chat_history[-1]['question']
            context_from_db = self.vectorstore_retriever.get_relevant_documents(current_question)
        else:
            context_from_db = self.vectorstore_retriever.get_relevant_documents("")

        # Convert the list of context documents into a string
        context_str = "\n".join([doc.page_content for doc in context_from_db])

        # Combine both chat history and the document context
        combined_context = f"Chat history:\n{chat_context}\n\nContext from the document:\n{context_str}"
        
        return combined_context


    def invoke(self, input_dict):
        question = input_dict.get("question")
        #here put a if-else that returns true or false depending on if question passes
        # guardrails checks
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"
        
        combined_context = self.build_combined_context()

        result = self.qa_chain.invoke({
            "query": question,
            "context": combined_context
        })

        self.update_chat_history(question, result['result'])
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"
        else:
          return result

    def guardrails(self, input):
        #if guardrails return true send back whatever the input is,
        #else send back an error message
        try:
            guard.validate(input)
            return True
        except Exception as e:
            return False


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import markdown

filepath = "./tutor_textbook.pdf"
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'docx', 'png', 'jpg', 'jpeg', 'gif'}
app = Flask(__name__)

@app.route("/",methods=["GET"])
def index():
   return render_template("index.html")

@app.route("/tutor-ai", methods=["GET", "POST"])
def tutor_ai():
    global url_data, prompt_data  # Access global variables

    if request.method == "POST":
        url_data = request.form.get("url")
        print("URL: ", url_data)
        if 'file' not in request.files:
            print('No file uploaded!')
        else:
          file = request.files['file']
          file.save(filepath)
          print("File saved:", filepath)
        if (url_data != ""):
            subprocess.check_call("curl", url_data, ">", "tutor_textbook.pdf")
        print("File: ",file)
        prompt_data = request.form.get("prompt")
        base_qa_pipeline = BaseQAPipeline()
        result = base_qa_pipeline.invoke({'question' : prompt_data})
        print(result)
        return render_template("tutor-ai.html", result=result)

    return render_template("tutor-ai.html")

@app.route('/how-it-works', methods=['GET'])
def how_it_works():
    return render_template('how-it-works.html')

@app.route('/generate-plan', methods=['GET', "POST"])
def generate_plan():
    if request.method == "POST":
        if 'file' not in request.files:
            print('No file uploaded!')
        else:
          file = request.files['file']
          file.save(filepath)
          print("File saved:", filepath)
        print("File: ",file)
        prompt_data = request.form.get("prompt")
        generate_plan = GenerateStudyPlan()
        result = generate_plan.invoke({'question' : prompt_data})
        result['result'] = markdown.markdown(result['result'])
        print(result)
        return render_template("generate-plan.html", result=result)

    return render_template("generate-plan.html")


if __name__ == "__main__":
    #from waitress import serve
    #serve(app, host="0.0.0.0", port=8081)
    # above code is for SERVER
    #below code right now is to debug
    print("Server is running...")
    app.run(port=8080)
    print("Stopping Server")

Server is running...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


In [ ]:
# compiled all code into one box

# Requires index.html template to be placed into templates/index.html to work


# Url: To input textbook url. Recognized as a .pdf format. Example url: https://www.mrbigler.com/downloads/Notes-Physics-1.pdf
# Prompt: To input question. Recognized as a string format. Example prompt: What is momentum?

# Response for now is given as a JSON response with question being your prompt and response being
# the tutor's answer.

# TO USE, PRESS THE LINK DOWN IN THE OUTPUT.
# please press CANCEL to the prompt it asks about restarting runtime to use new packages
# Final code with everything attached.
#Question Answering system using a pdf
#Install the libraries
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchain_openai
!pip install -q openai
!pip install -q langchain_core
!pip install -q pypdf
!pip install -q chromadb
!pip install -q Flask
!pip install -q markdown
!pip install -q langchain guardrails-ai
!pip install -q guardrails-ai

# configure with api key
!guardrails configure

# install guardrails libraries
!guardrails hub install hub://guardrails/ban_list
!guardrails hub install hub://guardrails/bias_check
!guardrails hub install hub://guardrails/nsfw_text
!guardrails hub install hub://guardrails/profanity_free
!guardrails hub install hub://guardrails/logic_check
!guardrails hub install hub://cartesia/mentions_drugs
!guardrails hub install hub://guardrails/politeness_check
!guardrails hub install hub://guardrails/toxic_language

##Imports for Baseline QA Pipeline
from langchain.document_loaders import PyPDFLoader # for loading the pdf
from langchain_openai import OpenAIEmbeddings # for creating embeddings
from langchain.vectorstores import Chroma # for the vectorization part
from langchain.chains import RetrievalQA #For the retrieval QA chain part # apparently deprecated
from langchain_openai import ChatOpenAI #for getting an LLM for QA chain
#from langchain_core.output_parsers import StrOutputParser #Not used currently, leaving, as can be used for parsing output from LLM
#from langchain_core.runnables import RunnablePassthrough #Not used currently, leaving, as can be used for getting LLM output
from langchain.prompts import ChatPromptTemplate #for setting up prompts


#Guardrails stuff
from guardrails import Guard, OnFailAction
from guardrails.hub import BanList, BiasCheck, NSFWText, ProfanityFree, LogicCheck, MentionsDrugs, PolitenessCheck#, ToxicLanguage# Updated import
#import guardrails.hub #which needs fuzzysearch, which is already downloaded
#from guardrails.datatypes import String
from langchain.chains import LLMChain
from langchain.llms import OpenAI
#Setup openai key
import os
import openai
from getpass import getpass
print("Please enter Open AI KEY")
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

from guardrails import Guard


guard = Guard().use_many(
    BiasCheck(
        threshold=0.5,
        on_fail="exception"
    ),

    NSFWText(
        threshold=0.8,
        validation_method="sentence"
    ),

    ProfanityFree(
        on_fail = "exception"
    ),

    LogicCheck(
        model="gpt-3.5-turbo",
        on_fail="exception"
    ),

    MentionsDrugs(
        on_fail = "exception"
    ),

    PolitenessCheck(
        llm_callable="gpt-3.5-turbo",
        on_fail = "exception"
    ),

    ToxicLanguage(
        threshold=0.5,
        validation_method="sentence",
        on_fail="exception"
    )

)

try:
  guard.validate("I hate you!")
  print("hello")
except Exception as e:
  print(e)
#Setup GenerateStudyPlan pipeline
class GenerateStudyPlan:
    def __init__(self):
        self.doc = "tutor_textbook.pdf"
        self.loader = PyPDFLoader(self.doc)

        # Load the document and store it in the 'data' variable
        self.data = self.loader.load_and_split()

        self.embeddings = OpenAIEmbeddings()
        self.vectordb = Chroma.from_documents(self.data, embedding=self.embeddings,
                                 persist_directory=".")

        # Initialize a language model with ChatOpenAI
        self.llm = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.6)

        #Setup a prompt template
        template = """\
            You are an assistant for generating study plans on a singular subject.

        Use the following pieces of retrieved context to answer the question.

        If the user has given a topic to study or topics that they need focus on,
        make the plan more focused on those topics.

        Give a nice and detailed study plan. If the user doesnt specify a type of 
        study plan (schedule wise), you can come up with your own, hourly or daily 
        plan, or you can ask the user to give the same instructions but with the 
        study plan of their choice. 

        Question: {question}

        Context: {context}

        Answer:

        """

        prompt = ChatPromptTemplate.from_template(template)

        chain_type_kwargs = {"prompt": prompt}


        # 1. Vectorstore-based retriever
        self.vectorstore_retriever = self.vectordb.as_retriever()

        # Initialize a RetrievalQA chain with the language model and vector database retriever
        self.qa_chain = RetrievalQA.from_chain_type(self.llm, retriever= self.vectorstore_retriever, chain_type_kwargs=chain_type_kwargs)
        self.chat_history = []  # Initialize chat history

    def update_chat_history(self, question, answer):
        self.chat_history.append({"question": question, "answer": answer})

    def build_combined_context(self):
        """Combine chat history and document context."""
        # Combine all previous chat history
        chat_context = "\n".join([f"Q: {entry['question']}\nA: {entry['answer']}" for entry in self.chat_history])
        
        # Fetch relevant context from the vector store based on the current question
        if self.chat_history:
            current_question = self.chat_history[-1]['question']
            context_from_db = self.vectorstore_retriever.get_relevant_documents(current_question)
        else:
            context_from_db = self.vectorstore_retriever.get_relevant_documents("")

        # Convert the list of context documents into a string
        context_str = "\n".join([doc.page_content for doc in context_from_db])

        # Combine both chat history and the document context
        combined_context = f"Chat history:\n{chat_context}\n\nContext from the document:\n{context_str}"
        
        return combined_context


    def invoke(self, input_dict):
        question = input_dict.get("question")
        #here put a if-else that returns true or false depending on if question passes
        # guardrails checks
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"
        
        combined_context = self.build_combined_context()

        result = self.qa_chain.invoke({
            "query": question,
            "context": combined_context
        })

        self.update_chat_history(question, result['result'])
        if (self.guardrails(question) == False):
          print("It has failed (this is only a message to debug)\n")
          return "Sorry, please ask another question"
        else:
          return result

    def guardrails(self, input):
        #if guardrails return true send back whatever the input is,
        #else send back an error message
        try:
            guard.validate(input)
            return True
        except Exception as e:
            return False

from flask import Flask, render_template, request, redirect, url_for
import markdown

filepath = "./tutor_textbook.pdf"
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'docx', 'png', 'jpg', 'jpeg', 'gif'}
app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def index():
    global url_data, prompt_data  # Access global variables

    if request.method == "POST":
        url_data = request.form.get("url")
        print("URL: ", url_data)
        if 'file' not in request.files:
            print('No file uploaded!')
        else:
          file = request.files['file']
          file.save(filepath)
          print("File saved:", filepath)
        if (url_data != ""):
            !curl {url_data} > tutor_textbook.pdf
        print("File: ",file)
        prompt_data = request.form.get("prompt")
        base_qa_pipeline = BaseQAPipeline()
        result = base_qa_pipeline.invoke({'question' : prompt_data})
        print(result)
        return render_template("index.html", result=result)

    return render_template("index.html")

@app.route('/how-it-works', methods=['GET'])
def how_it_works():
    return render_template('how-it-works.html')

@app.route('/generate-plan', methods=['GET', "POST"])
def generate_plan():
    if request.method == "POST":
        if 'file' not in request.files:
            print('No file uploaded!')
        else:
          file = request.files['file']
          file.save(filepath)
          print("File saved:", filepath)
        print("File: ",file)
        prompt_data = request.form.get("prompt")
        generate_plan = GenerateStudyPlan()
        result = generate_plan.invoke({'question' : prompt_data})
        result['result'] = markdown.markdown(result['result'])
        print(result)
        return render_template("generate-plan.html", result=result)

    return render_template("generate-plan.html")


if __name__ == "__main__":


    app.run(port=5001)
# Requires index.html template to be placed into templates/index.html to work


# Url: To input textbook url. Recognized as a .pdf format. Example url: https://www.mrbigler.com/downloads/Notes-Physics-1.pdf
# Prompt: To input question. Recognized as a string format. Example prompt: What is momentum?

# Response for now is given as a JSON response with question being your prompt and response being
# the tutor's answer.

# TO USE, PRESS THE LINK DOWN IN THE OUTPUT.
# please press CANCEL to the prompt it asks about restarting runtime to use new packages


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.4 MB/s eta 0:00:

Please enter Open AI KEY
··········
https://qg8uz4v9w7d-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-1-4ab8419cdd29>", line 119, in index
    return render_template("index.html")
  File "/usr/local/lib/python3.10/dist-packages/flask